In [78]:
import bz2
import json
import requests as rq
import time

In [79]:
import lzma

In [98]:
dump = "/10TBdrive/datashare/Reddit/submissions/RS_2017-02.bz2"
# dump = "/10TBdrive/datashare/Reddit/submissions/RS_2017-12.xz"

In [99]:
openedDump = bz2.BZ2File(dump, "r")
# openedDump = lzma.open(dump)

In [100]:
count = 0
for line in openedDump:
    if count > 10:
        break
    print(repr(line))
    count+=1

b'{"contest_mode":false,"subreddit_id":"t5_2qwwd","url":"https://i.reddituploads.com/572369dcd4ba4c09a2469699b541d0c2?fit=max&amp;h=1536&amp;w=1536&amp;s=cd34212ff8b47a01a272ce361df1bb08","subreddit":"Honda","archived":false,"selftext":"","permalink":"/r/Honda/comments/5rc1f2/the_new_s2000/","media":null,"author_flair_css_class":null,"suggested_sort":null,"score":90,"link_flair_text":null,"is_self":false,"preview":{"images":[{"resolutions":[{"height":60,"width":108,"url":"https://i.redditmedia.com/B3QKyDOIh83D7ergRLCM96dQJqvwS9DQL5GS9i-LYaI.jpg?fit=crop&amp;crop=faces%2Centropy&amp;arh=2&amp;w=108&amp;s=748620ea3d18e5a2469cddcc6aedea7f"},{"width":216,"url":"https://i.redditmedia.com/B3QKyDOIh83D7ergRLCM96dQJqvwS9DQL5GS9i-LYaI.jpg?fit=crop&amp;crop=faces%2Centropy&amp;arh=2&amp;w=216&amp;s=c169b1567750606290557e235aa5f67f","height":121},{"height":180,"width":320,"url":"https://i.redditmedia.com/B3QKyDOIh83D7ergRLCM96dQJqvwS9DQL5GS9i-LYaI.jpg?fit=crop&amp;crop=faces%2Centropy&amp;arh=2&a

In [101]:
def getJson(url):
    session_headers = {
    'Host': 'www.reddit.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #'Referer': 'http://localhost:9000/notebooks/Reddit/Notebooks/SuicideWatchSubreddit.ipynb'
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
    'Cookie': "loid=00000000000gslzvdi.2.1507591930964.Z0FBQUFBQlozQWI3M243MTdFMUgzNWtQNzNVbzg5Z3NXYmxpLUdRTWVXdEpWQzBfbkxBSVNNVVF3dE1zQlN5cVMzeGtxZXlmT0JQY2JvTld2NWlVeGlGeFpkNnBYM2ZIYlNNR2xMWGFFS3hieDZpWEJ5OVlyTng0MUFZZjNYdjQ0ZURFNTFlSmlVRGw; edgebucket=T8nkTF0z7WPnNzlE8B; __gads=ID=ab9c1b0fcb7eaefc:T=1507591936:S=ALNI_Mao2tvEoyJzEDEuxxhfZBlOQHXlsA; eu_cookie_v2=3; _ga=GA1.2.1088810883.1507591931; __utma=55650728.1088810883.1507591931.1507912304.1507912553.14; __utmz=55650728.1507912553.14.11.utmcsr=reddit|utmccn=(not%20set)|utmcmd=hot|utmcct=comments; aa=1; _recentclicks2=t3_7cfzsx%2Ct3_7cfw08%2Ct3_7cdofu%2Ct3_7caog1%2Ct3_7cdvud; _recent_srs=t5_2qpzs%2Ct5_2qh1q%2Ct5_2qh49%2Ct5_2rtff%2Ct5_36buk%2Ct5_2r7yd%2Ct5_2rks3%2Ct5_3j2jr%2Ct5_2sumw%2Ct5_2xp2o; pc=r3; session_tracker=tPgx9U9yPt99jYRDqC.0.1510585492994.Z0FBQUFBQmFDYlNWcWZHclI5WWxDWW9kQ0dURk9FRU9BZjFJTURWZElzaldRbkROaEJUOWE1czRQUWxPUzUwOFFwY3R3ckI5VkNDUlhJc01NVEtEeHVWd25CdUpXWEI1X012RkJaS3cyNXdNbk5vOE9DSXp5ZlhuTzF3a2pxT2dLZ3laMDhtQlhib3k; initref=localhost"
    }

    client = rq.session()
    resp = client.get(url , headers=session_headers)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print("Failed to get page")
        return None
def getSubredditId(json):
    if type(json) == dict:
        sub_id = json['data']['children'][0]['data']['subreddit_id']
        return sub_id
    else: 
        print("Invalid Format")
        return None

def makeSubUrl(name):
    return "https://www.reddit.com/r/" + name + "/.json"

def getSubIds(RedditList):
    subIds = {}
    for name in RedditList:
        json = getJson(makeSubUrl(name))
        print (getSubredditId(json))
        subIds[name] = getSubredditId(json)
    return subIds

def filterDumpbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit'] in subID_dict:
            if lineData['subreddit'] not in returnData:
                returnData[lineData['subreddit']] = []
            postId = lineData['link_id'].strip().split('_')[1]
            
            if postId not in returnData[lineData['subreddit']]:
                returnData[lineData['subreddit']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData


def filterSubmissionsbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit_id'] in subID_dict:
            if lineData['subreddit_id'] not in returnData:
                returnData[lineData['subreddit_id']] = []
            postId = lineData['id']
            
            if postId not in returnData[lineData['subreddit_id']]:
                returnData[lineData['subreddit_id']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData



def getSubmissionPermalinks(filePath , comments_Thresh = 20 , limit=None):
    import json
    import bz2
    import lzma
    
    ext = filePath.split('.')[-1]
    
    if ext == 'bz2':
        openedDump = bz2.BZ2File(filePath, "r")
    elif ext == 'xz':
        openedDump = lzma.open(dump)
    else:
        print ("Invalid file type")
    count = 0
    returnData = {}
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if lineData['num_comments'] > comments_Thresh:
            if (limit!=None and count > limit):
                return returnData
            try:
                returnData[lineData['id']] = lineData['permalink']
            except:
                print("Permalink not found, moving on!")
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData

In [102]:
subreddits = ["bipolar" , "BipolarReddit" , "BipolarSOs" , "Health" , "addiction", "depression","SuicideWatch","Anxiety","Chicago"] 

In [103]:
subID_dict = getSubIds(subreddits)

t5_2qnv4
t5_2qzru
t5_2zww6
t5_2qh9z
t5_2qwvc
t5_2qqqf
t5_2qpzs
t5_2qmij
t5_2qh2t


In [104]:
subID_dict

{'Anxiety': 't5_2qmij',
 'BipolarReddit': 't5_2qzru',
 'BipolarSOs': 't5_2zww6',
 'Chicago': 't5_2qh2t',
 'Health': 't5_2qh9z',
 'SuicideWatch': 't5_2qpzs',
 'addiction': 't5_2qwvc',
 'bipolar': 't5_2qnv4',
 'depression': 't5_2qqqf'}

In [105]:
# postData = filterDumpbySub(dump,subID_dict)
postData = getSubmissionPermalinks(dump,comments_Thresh= 10)

=============Parsing============
{'bipolar': 't5_2qnv4', 'BipolarReddit': 't5_2qzru', 'BipolarSOs': 't5_2zww6', 'Health': 't5_2qh9z', 'addiction': 't5_2qwvc', 'depression': 't5_2qqqf', 'SuicideWatch': 't5_2qpzs', 'Anxiety': 't5_2qmij', 'Chicago': 't5_2qh2t'}
Done parsing %d posts in %d seconds (0, 0.04129910469055176)
Done parsing %d posts in %d seconds (100000, 9.788041114807129)
Done parsing %d posts in %d seconds (200000, 9.797199487686157)
Done parsing %d posts in %d seconds (300000, 10.040953159332275)
Done parsing %d posts in %d seconds (400000, 9.609859228134155)
Done parsing %d posts in %d seconds (500000, 9.667799472808838)
Done parsing %d posts in %d seconds (600000, 9.889967203140259)
Done parsing %d posts in %d seconds (700000, 9.680508852005005)
Done parsing %d posts in %d seconds (800000, 9.499156951904297)
Done parsing %d posts in %d seconds (900000, 9.932097911834717)
Done parsing %d posts in %d seconds (1000000, 9.732749700546265)
Done parsing %d posts in %d seconds (1

In [106]:
len(postData)

1126118

In [107]:
import pickle as pkl
pkl.dump(file=open('/10TBdrive/datashare/Reddit/Submissions022017_atleast10.pkl','wb'),obj=postData)

In [97]:
postData[list(postData.keys())[1]]

'/r/nba/comments/5lcgjd/game_thread_new_york_knicks_1616_houston_rockets/'

In [31]:
# import pickle as pkl
# with open("/datasets_1/sagarj/IoPPN_collab/MentalHealth_2017_posts.pkl",'w') as f:
#     pkl.dump(postData,f)

In [32]:
len(list(set(postData['depression'])))

KeyError: 'depression'

In [ ]:
# import pickle as pkl
# with open("ChangemyView2017.pkl",'w') as f:
#     pkl.dump(postData['changemyview'],f)

In [ ]:
# with open("Filtered_posts_2017.json",'w') as f:
#     json.dump(postData,f)